## PDF Query Using Langchain

### In this we used pdf containing some fake data of alkaaf forest that doesn't exit in real life. Fine tune LLM on this pdf using Langchain framework 

In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

   ---------------------------------------- 0.0/798.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/798.7 kB ? eta -:--:--
    --------------------------------------- 10.2/798.7 kB ? eta -:--:--
    --------------------------------------- 10.2/798.7 kB ? eta -:--:--
    --------------------------------------- 10.2/798.7 kB ? eta -:--:--
   - ------------------------------------- 30.7/798.7 kB 145.2 kB/s eta 0:00:06
   - ------------------------------------- 30.7/798.7 kB 145.2 kB/s eta 0:00:06
   --- ----------------------------------- 61.4/798.7 kB 217.9 kB/s eta 0:00:04
   --- ----------------------------------- 71.7/798.7 kB 231.0 kB/s eta 0:00:04
   ----- -------------------------------- 112.6/798.7 kB 312.2 kB/s eta 0:00:03
   ------ ------------------------------- 143.4/798.7 kB 369.8 kB/s eta 0:00:02
   --------- ---------------------------- 194.6/798.7 kB 454.0 kB/s eta 0:00:02
   ---------- --------------------------- 225.3/798.7 kB 491.0 kB/s eta 0:00:02
  

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "enter ur openai api key"

In [4]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('Forest.pdf')

In [5]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [6]:
raw_text

"The Alkaf Forest, located in the picturesque state of Lahore, stands as a testament to the natural \nbeauty and ecological richness of the region. Stretching an impressive 100 meters in length, this \nforest boasts a diverse array of flora and fauna, making it a vital ecosystem within the state. Nestled \namidst rolling hi lls and verdant landscapes, the Alkaf Forest serves as a sanctuary for numerous plant \nand animal species, providing essential habitat and refuge in an otherwise rapidly developing area.  \nWithin the lush canopy of the Alkaf Forest, visitors can discover a stunning variety of plant life, from \ntowering trees to delicate wildflowers. The dense foliage creates a cool and tranquil environment, \ninviting exploration and contemplation. Strolling al ong the forest trails, one can encounter majestic \noak, maple, and pine trees, their branches reaching skyward in a majestic display of nature's \ngrandeur. Beneath the canopy, a tapestry of ferns, mosses, and lichens car

In [7]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [8]:
len(texts)

4

In [9]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

C:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
document_search = FAISS.from_texts(texts, embeddings)

In [11]:
document_search


In [12]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [13]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

C:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [14]:
query = "where is alkaaf forest located"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

C:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' The Alkaf Forest is located in the state of Lahore.'

In [15]:
query = "write 2 sentences about alkaaf forest"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The Alkaf Forest, located in Lahore, is a vital ecosystem with diverse flora and fauna, serving as a natural filtration system and carbon sink. It also offers recreational and cultural value for visitors seeking solace in its tranquil atmosphere and scenic beauty.'